In [1]:
#Basic Ad Analysis
## Ramya Prabhakar

In [31]:
#import the required packages

import pandas as pd
import arrow
from sklearn import linear_model
# import stats models for the reg statistics
import statsmodels.api as sm
import warnings

In [32]:
#ignore warnings
warnings.filterwarnings("ignore")

In [33]:
# Data uploaded in github
# Read the travel pony facebook csv file using pandas
df = pd.read_csv('Travel Pony Facebook.csv')

In [34]:
#check the number of rows and columns in the dataframe
df.count

<bound method DataFrame.count of      Start Date  End Date                                      Campaign Name  \
0       12/2/13   12/2/13                                      Cyber Weekend   
1       12/1/13   12/1/13                                      Cyber Weekend   
2      11/30/13  11/30/13                                      Cyber Weekend   
3      11/29/13  11/29/13                                      Cyber Weekend   
4      11/22/13  11/22/13                            TravelPony: General Ads   
5      11/22/13  11/22/13                        TravelPony: Conversion Test   
6      11/21/13  11/21/13                        TravelPony: Conversion Test   
7      11/21/13  11/21/13                            TravelPony: General Ads   
8      11/20/13  11/20/13                        TravelPony: Conversion Test   
9      11/20/13  11/20/13                            TravelPony: General Ads   
10     11/19/13  11/19/13                        TravelPony: Conversion Test   
11     

In [35]:
# create a new column CPI and calculate the same using amount spent/impressions
df['CPI'] = df['Amount Spent (USD)']/df['Impressions']

In [36]:
#create an empty list to note down the day of the week
dayOfWeek = []

In [37]:
#Extract the day of the weeek as a numerical value from the start date field and store it in the list dayOfWeek
for i,row in df.iterrows():
    dt = arrow.get(row['Start Date'], 'M/D/YY')
    x = dt.isoweekday()
    dayOfWeek.append(x)

In [38]:
#create a new column in the dataframe and store the day of the week corresponding to the date
df['dayOfWeek']=pd.Series(dayOfWeek)

In [39]:
#Create a pivot table to sum the number of times each day of the week is repeated
pivot_dayOfWeek = df.pivot_table(index='dayOfWeek', aggfunc='sum')

In [40]:
#Calculate the best and worst days of the week based on the max and the min CPI    
dw=pd.DataFrame(pivot_dayOfWeek,columns=['CPI','dayOfWeek']) 
print("Best day of the week:",dw['CPI'].idxmin(),";CPI",dw['CPI'][dw['CPI'].idxmin()])
print("Worst day of the week:",dw['CPI'].idxmax(),";CPI",dw['CPI'][dw['CPI'].idxmax()])

Best day of the week: 6 ;CPI 1.3117197697515264
Worst day of the week: 5 ;CPI 2.05663914118258


In [41]:
#compute the correlation between Amount Spent and the following variables:
#- Reach
#- Frequency 
#- Unique Clicks
#- Page Likes

data_corr=df.corr()
reach_amtSpent=data_corr['Reach']['Amount Spent (USD)']
freq_amtSpent=data_corr['Frequency']['Amount Spent (USD)']
unqClicks_amtSpent=data_corr['Unique Clicks']['Amount Spent (USD)']
pgLikes_amtSpent=data_corr['Page Likes']['Amount Spent (USD)']

data={'variable':['Reach','Frequency','Unique Clicks','Page Likes'],'correlation':[reach_amtSpent,freq_amtSpent,unqClicks_amtSpent,pgLikes_amtSpent]}
corr_df=pd.DataFrame(data,columns=['variable','correlation'])

In [42]:
# Which correlation is the strongest? What does that mean practically? 
# (respond in a tweet or less)
print("Amount Spent (USD) has the strongest Correlation with ",corr_df['variable'][corr_df['correlation'].idxmax()],"=",corr_df['correlation'][corr_df['correlation'].idxmax()])


Amount Spent (USD) has the strongest Correlation with  Unique Clicks = 0.8829931774784341


In [43]:
# Finally, perform a simple multiple regression analysis where 
# Unique Clicks is the dependent variable and Reach and Frequency are the independent
# (predictor) variables.
# What variable most strongly predicts unique clicks? 
# What does that mean practically? (respond in a tweet or less)

In [44]:
#use the linear_model from sklearn
df = pd.read_csv('Travel Pony Facebook.csv')

x1=df[['Reach','Frequency']]
y1=df['Unique Clicks']
regr = linear_model.LinearRegression()
regr.fit(x1, y1)

print ("Using Linear_model from sklearn")
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
    
#predicted Y for the regression model
predictions = regr.predict(x1)
print(predictions)

# r2 value
print(" R squared value is ", regr.score(x1,y1))

Using Linear_model from sklearn
Intercept: 
 29.227730390224764
Coefficients: 
 [ 2.06474936e-03 -2.50632823e+01]
[545.46559791 433.95502979 519.77018255 ...   4.39363526   4.45970724
  11.70488637]
 R squared value is  0.534339731180361


In [46]:
#now use the statsmodels to confirm the findings
      
X2 = sm.add_constant(x1)
print (X2)
est = sm.OLS(y1, X2)
print (est)
est2 = est.fit()
print(est2.summary()) 

X = df[['Reach', 'Frequency']]
y = df['Unique Clicks']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

print ("\nThe variable Frequency strongly predicts the number of Unique clicks since it has a greater co efficient!")

      const   Reach  Frequency
0       1.0  278117   2.314303
1       1.0  221255   2.079090
2       1.0  267915   2.499069
3       1.0  280332   2.365406
4       1.0   14269   1.000420
5       1.0    7096   1.018743
6       1.0   29437   1.000000
7       1.0   30320   1.018239
8       1.0   25951   1.026820
9       1.0   49038   1.000000
10      1.0   32851   1.023287
11      1.0   53974   1.000000
12      1.0   81149   1.004128
13      1.0   81237   1.000000
14      1.0   68209   1.000000
15      1.0   45828   1.000000
16      1.0    8466   1.018663
17      1.0    1111   1.001800
18      1.0    5043   1.008527
19      1.0   10825   1.017644
20      1.0    3384   1.002660
21      1.0    8054   1.001738
22      1.0   19465   1.000000
23      1.0    6698   1.016124
24      1.0   15852   1.020250
25      1.0   21453   1.019391
26      1.0    8449   1.010179
27      1.0   19738   1.000000
28      1.0   16716   1.000000
29      1.0   13166   1.000000
...     ...     ...        ...
3675    